In [1]:
import findspark
findspark.init()
import pyspark
sc=pyspark.SparkContext(appName="case_study_pyspark")

In [2]:
import pyspark.sql as pss
spark = pss.SparkSession.builder.appName("case_study_pyspark").getOrCreate()

In [3]:
slines = sc.textFile(r"C:\Users\apurba.mondal\Documents\sales_CS.txt")

In [4]:
slines.take(10)

['1,6,11,2022-01-01,10,sghdfkrui',
 '2,9,12,2022-01-01,50,yuehtskam',
 '3,10,13,2022-01-02,20,wuiodfgfj',
 '4,7,14,2022-01-02,10,operdfvms',
 '5,8,15,2022-01-03,30,puerwyysb',
 '6,10,13,2022-01-03,15,sjdjhhjkj',
 '7,6,11,2022-01-04,5,dwjdqdmkk',
 '8,7,14,2022-01-04,15,cxvxnmndk',
 '9,8,15,2022-01-05,20,reyokqdklk',
 '10,9,12,2022-01-05,40,qplsmdgf']

In [5]:
sfield = slines.map(lambda l: l.split(","))

In [6]:
sfield.take(5)

[['1', '6', '11', '2022-01-01', '10', 'sghdfkrui'],
 ['2', '9', '12', '2022-01-01', '50', 'yuehtskam'],
 ['3', '10', '13', '2022-01-02', '20', 'wuiodfgfj'],
 ['4', '7', '14', '2022-01-02', '10', 'operdfvms'],
 ['5', '8', '15', '2022-01-03', '30', 'puerwyysb']]

In [7]:
rdd_sales = sfield.map(lambda p : pss.Row(order_id=int(p[0]),prod_id=int(p[1]),seller_id=int(p[2]),date=p[3],amount=int(p[4]),bill_text=p[5]))

In [8]:
rdd_sales.take(5)

[Row(order_id=1, prod_id=6, seller_id=11, date='2022-01-01', amount=10, bill_text='sghdfkrui'),
 Row(order_id=2, prod_id=9, seller_id=12, date='2022-01-01', amount=50, bill_text='yuehtskam'),
 Row(order_id=3, prod_id=10, seller_id=13, date='2022-01-02', amount=20, bill_text='wuiodfgfj'),
 Row(order_id=4, prod_id=7, seller_id=14, date='2022-01-02', amount=10, bill_text='operdfvms'),
 Row(order_id=5, prod_id=8, seller_id=15, date='2022-01-03', amount=30, bill_text='puerwyysb')]

In [9]:
sales_df = spark.createDataFrame(rdd_sales)

In [10]:
sales_df.show()

+--------+-------+---------+----------+------+----------+
|order_id|prod_id|seller_id|      date|amount| bill_text|
+--------+-------+---------+----------+------+----------+
|       1|      6|       11|2022-01-01|    10| sghdfkrui|
|       2|      9|       12|2022-01-01|    50| yuehtskam|
|       3|     10|       13|2022-01-02|    20| wuiodfgfj|
|       4|      7|       14|2022-01-02|    10| operdfvms|
|       5|      8|       15|2022-01-03|    30| puerwyysb|
|       6|     10|       13|2022-01-03|    15| sjdjhhjkj|
|       7|      6|       11|2022-01-04|     5| dwjdqdmkk|
|       8|      7|       14|2022-01-04|    15| cxvxnmndk|
|       9|      8|       15|2022-01-05|    20|reyokqdklk|
|      10|      9|       12|2022-01-05|    40|  qplsmdgf|
+--------+-------+---------+----------+------+----------+



In [11]:
sales_df.printSchema()

root
 |-- order_id: long (nullable = true)
 |-- prod_id: long (nullable = true)
 |-- seller_id: long (nullable = true)
 |-- date: string (nullable = true)
 |-- amount: long (nullable = true)
 |-- bill_text: string (nullable = true)



In [12]:
plines = sc.textFile(r"C:\Users\apurba.mondal\Documents\product_CS.txt")
pfield = plines.map(lambda l : l.split(","))
rdd_prod = pfield.map(lambda p : pss.Row(prodid=int(p[0]),prod_name=p[1],price=float(p[2])))
product_df = spark.createDataFrame(rdd_prod)
product_df.show()
product_df.printSchema()

+------+---------------+-------+
|prodid|      prod_name|  price|
+------+---------------+-------+
|     7|washing machine|25499.0|
|     9|     micro oven| 2749.0|
|     8|        almirah| 7770.0|
|    10|   dining table|15999.0|
|     6|          couch|20499.0|
+------+---------------+-------+

root
 |-- prodid: long (nullable = true)
 |-- prod_name: string (nullable = true)
 |-- price: double (nullable = true)



In [13]:
sslines = sc.textFile(r"C:\Users\apurba.mondal\Documents\seller_CS.txt")
ssfield = sslines.map(lambda l : l.split(","))
rdd_seller = ssfield.map(lambda p : pss.Row(sellerid=int(p[0]),seller_name=p[1],daily_target=int(p[2])))
seller_df = spark.createDataFrame(rdd_seller)
seller_df.show()
seller_df.printSchema()

+--------+-----------+------------+
|sellerid|seller_name|daily_target|
+--------+-----------+------------+
|      12|     apurba|          30|
|      15|   anukriti|          35|
|      11|    anushka|          10|
|      13|      zidan|          30|
|      14|    anubhaw|          15|
+--------+-----------+------------+

root
 |-- sellerid: long (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- daily_target: long (nullable = true)



In [14]:
new_df = sales_df.join(product_df,sales_df.prod_id==product_df.prodid,'outer')
new_df.show()

+--------+-------+---------+----------+------+----------+------+---------------+-------+
|order_id|prod_id|seller_id|      date|amount| bill_text|prodid|      prod_name|  price|
+--------+-------+---------+----------+------+----------+------+---------------+-------+
|       1|      6|       11|2022-01-01|    10| sghdfkrui|     6|          couch|20499.0|
|       7|      6|       11|2022-01-04|     5| dwjdqdmkk|     6|          couch|20499.0|
|       4|      7|       14|2022-01-02|    10| operdfvms|     7|washing machine|25499.0|
|       8|      7|       14|2022-01-04|    15| cxvxnmndk|     7|washing machine|25499.0|
|       5|      8|       15|2022-01-03|    30| puerwyysb|     8|        almirah| 7770.0|
|       9|      8|       15|2022-01-05|    20|reyokqdklk|     8|        almirah| 7770.0|
|       2|      9|       12|2022-01-01|    50| yuehtskam|     9|     micro oven| 2749.0|
|      10|      9|       12|2022-01-05|    40|  qplsmdgf|     9|     micro oven| 2749.0|
|       3|     10|   

In [15]:
new_df = new_df.join(seller_df,new_df.seller_id==seller_df.sellerid,'outer')
new_df.show()

+--------+-------+---------+----------+------+----------+------+---------------+-------+--------+-----------+------------+
|order_id|prod_id|seller_id|      date|amount| bill_text|prodid|      prod_name|  price|sellerid|seller_name|daily_target|
+--------+-------+---------+----------+------+----------+------+---------------+-------+--------+-----------+------------+
|       1|      6|       11|2022-01-01|    10| sghdfkrui|     6|          couch|20499.0|      11|    anushka|          10|
|       7|      6|       11|2022-01-04|     5| dwjdqdmkk|     6|          couch|20499.0|      11|    anushka|          10|
|       2|      9|       12|2022-01-01|    50| yuehtskam|     9|     micro oven| 2749.0|      12|     apurba|          30|
|      10|      9|       12|2022-01-05|    40|  qplsmdgf|     9|     micro oven| 2749.0|      12|     apurba|          30|
|       3|     10|       13|2022-01-02|    20| wuiodfgfj|    10|   dining table|15999.0|      13|      zidan|          30|
|       6|     1

In [16]:
final_df = new_df.select("order_id","date","prod_id","prod_name","price","amount","seller_id","seller_name","daily_target",\
                         "bill_text")
final_df.show()
final_df.printSchema()

+--------+----------+-------+---------------+-------+------+---------+-----------+------------+----------+
|order_id|      date|prod_id|      prod_name|  price|amount|seller_id|seller_name|daily_target| bill_text|
+--------+----------+-------+---------------+-------+------+---------+-----------+------------+----------+
|       1|2022-01-01|      6|          couch|20499.0|    10|       11|    anushka|          10| sghdfkrui|
|       7|2022-01-04|      6|          couch|20499.0|     5|       11|    anushka|          10| dwjdqdmkk|
|       2|2022-01-01|      9|     micro oven| 2749.0|    50|       12|     apurba|          30| yuehtskam|
|      10|2022-01-05|      9|     micro oven| 2749.0|    40|       12|     apurba|          30|  qplsmdgf|
|       3|2022-01-02|     10|   dining table|15999.0|    20|       13|      zidan|          30| wuiodfgfj|
|       6|2022-01-03|     10|   dining table|15999.0|    15|       13|      zidan|          30| sjdjhhjkj|
|       4|2022-01-02|      7|washing 

###  Find out how many orders, how many products and how many sellers are in the data

In [17]:
import pyspark.sql.functions as psf
df1 = final_df.select(psf.countDistinct("order_id")).withColumnRenamed("count(DISTINCT order_id)","total_no_of_orders")
df2 = final_df.select(psf.countDistinct("prod_id")).withColumnRenamed("count(DISTINCT prod_id)","total_no_of_products")
df3 = final_df.select(psf.countDistinct("seller_id")).withColumnRenamed("count(DISTINCT seller_id)","total_no_of_sellers")
df1.show()
df2.show()
df3.show()

+------------------+
|total_no_of_orders|
+------------------+
|                10|
+------------------+

+--------------------+
|total_no_of_products|
+--------------------+
|                   5|
+--------------------+

+-------------------+
|total_no_of_sellers|
+-------------------+
|                  5|
+-------------------+



### How many products have been sold at least once?

In [18]:
df4 = final_df.filter(final_df["amount"] > 0).select(psf.countDistinct("prod_id")).\
withColumnRenamed("count(DISTINCT prod_id)","no_of_products_sold_atleast_once")
df4.show()

+--------------------------------+
|no_of_products_sold_atleast_once|
+--------------------------------+
|                               5|
+--------------------------------+



### Which product has been ordered most??? 

In [24]:
df5 = final_df.groupBy("prod_name").agg({"amount":"sum"}).orderBy(psf.col("sum(amount)").desc()).\
withColumnRenamed("sum(amount)","total_sales")
df5.show(1)

+----------+-----------+
| prod_name|total_sales|
+----------+-----------+
|micro oven|         90|
+----------+-----------+
only showing top 1 row



### How many distinct products have been sold in each day? 

In [32]:
df6 = final_df.groupBy("date","prod_name").agg({"amount":"sum"}).orderBy(psf.col("date")).\
withColumnRenamed("sum(amount)","sales_on_2022-01-01").where(final_df.date=='2022-01-01').\
select("prod_name","sales_on_2022-01-01")
df6.show()
df6 = final_df.groupBy("date","prod_name").agg({"amount":"sum"}).orderBy(psf.col("date")).\
withColumnRenamed("sum(amount)","sales_on_2022-01-02").where(final_df.date=='2022-01-02').\
select("prod_name","sales_on_2022-01-02")
df6.show()
df6 = final_df.groupBy("date","prod_name").agg({"amount":"sum"}).orderBy(psf.col("date")).\
withColumnRenamed("sum(amount)","sales_on_2022-01-03").where(final_df.date=='2022-01-03').\
select("prod_name","sales_on_2022-01-03")
df6.show()
df6 = final_df.groupBy("date","prod_name").agg({"amount":"sum"}).orderBy(psf.col("date")).\
withColumnRenamed("sum(amount)","sales_on_2022-01-04").where(final_df.date=='2022-01-04').\
select("prod_name","sales_on_2022-01-04")
df6.show()
df6 = final_df.groupBy("date","prod_name").agg({"amount":"sum"}).orderBy(psf.col("date")).\
withColumnRenamed("sum(amount)","sales_on_2022-01-05").where(final_df.date=='2022-01-05').\
select("prod_name","sales_on_2022-01-05")
df6.show()

+----------+-------------------+
| prod_name|sales_on_2022-01-01|
+----------+-------------------+
|micro oven|                 50|
|     couch|                 10|
+----------+-------------------+

+---------------+-------------------+
|      prod_name|sales_on_2022-01-02|
+---------------+-------------------+
|   dining table|                 20|
|washing machine|                 10|
+---------------+-------------------+

+------------+-------------------+
|   prod_name|sales_on_2022-01-03|
+------------+-------------------+
|     almirah|                 30|
|dining table|                 15|
+------------+-------------------+

+---------------+-------------------+
|      prod_name|sales_on_2022-01-04|
+---------------+-------------------+
|washing machine|                 15|
|          couch|                  5|
+---------------+-------------------+

+----------+-------------------+
| prod_name|sales_on_2022-01-05|
+----------+-------------------+
|   almirah|                 20|


### What is the average revenue of the orders???

In [39]:
final_df = final_df.withColumn("revenue",psf.col("price")*psf.col("amount"))
final_df.show()

+--------+----------+-------+---------------+-------+------+---------+-----------+------------+----------+--------+
|order_id|      date|prod_id|      prod_name|  price|amount|seller_id|seller_name|daily_target| bill_text| revenue|
+--------+----------+-------+---------------+-------+------+---------+-----------+------------+----------+--------+
|       1|2022-01-01|      6|          couch|20499.0|    10|       11|    anushka|          10| sghdfkrui|204990.0|
|       7|2022-01-04|      6|          couch|20499.0|     5|       11|    anushka|          10| dwjdqdmkk|102495.0|
|       2|2022-01-01|      9|     micro oven| 2749.0|    50|       12|     apurba|          30| yuehtskam|137450.0|
|      10|2022-01-05|      9|     micro oven| 2749.0|    40|       12|     apurba|          30|  qplsmdgf|109960.0|
|       3|2022-01-02|     10|   dining table|15999.0|    20|       13|      zidan|          30| wuiodfgfj|319980.0|
|       6|2022-01-03|     10|   dining table|15999.0|    15|       13|  

In [42]:
df7 = final_df.select(psf.avg("revenue")).withColumnRenamed("avg(revenue)","average revenue of orders")
df7.show()

+-------------------------+
|average revenue of orders|
+-------------------------+
|                 214083.5|
+-------------------------+



### Who is the second most selling persons among sellers(revenue wise)???

In [66]:
df8 = final_df.groupBy("seller_name").agg({"revenue":"sum"}).withColumnRenamed("sum(revenue)","total_revenue")

In [67]:
df8.createOrReplaceTempView("revenue")
df9 = spark.sql("select seller_name,total_revenue as 2nd_highest_revenue from revenue where total_revenue = \
(select max(total_revenue) as from revenue where total_revenue <>(select max(total_revenue) from revenue))")
df9.show()

+-----------+-------------------+
|seller_name|2nd_highest_revenue|
+-----------+-------------------+
|      zidan|           559965.0|
+-----------+-------------------+



### Who is the lowest selling persons among sellers(revenue wise)???

In [68]:
df10 = spark.sql("select seller_name,total_revenue as lowest_revenue from revenue where total_revenue= \
(select min(total_revenue) from revenue)")
df10.show()

+-----------+--------------+
|seller_name|lowest_revenue|
+-----------+--------------+
|     apurba|      247410.0|
+-----------+--------------+



### For each seller, what is the percentage contribution of an order to the seller’s daily quota ???

In [74]:
final_df = final_df.withColumn("percent_contribution",psf.round((psf.col("amount")/psf.col("daily_target"))*100,2))
final_df.show()

+--------+----------+-------+---------------+-------+------+---------+-----------+------------+----------+--------+--------------------+
|order_id|      date|prod_id|      prod_name|  price|amount|seller_id|seller_name|daily_target| bill_text| revenue|percent_contribution|
+--------+----------+-------+---------------+-------+------+---------+-----------+------------+----------+--------+--------------------+
|       1|2022-01-01|      6|          couch|20499.0|    10|       11|    anushka|          10| sghdfkrui|204990.0|               100.0|
|       7|2022-01-04|      6|          couch|20499.0|     5|       11|    anushka|          10| dwjdqdmkk|102495.0|                50.0|
|       2|2022-01-01|      9|     micro oven| 2749.0|    50|       12|     apurba|          30| yuehtskam|137450.0|              166.67|
|      10|2022-01-05|      9|     micro oven| 2749.0|    40|       12|     apurba|          30|  qplsmdgf|109960.0|              133.33|
|       3|2022-01-02|     10|   dining ta

In [75]:
df11 = final_df.select("date","seller_name","daily_target","percent_contribution")
df11.show()

+----------+-----------+------------+--------------------+
|      date|seller_name|daily_target|percent_contribution|
+----------+-----------+------------+--------------------+
|2022-01-01|    anushka|          10|               100.0|
|2022-01-04|    anushka|          10|                50.0|
|2022-01-01|     apurba|          30|              166.67|
|2022-01-05|     apurba|          30|              133.33|
|2022-01-02|      zidan|          30|               66.67|
|2022-01-03|      zidan|          30|                50.0|
|2022-01-02|    anubhaw|          15|               66.67|
|2022-01-04|    anubhaw|          15|               100.0|
|2022-01-03|   anukriti|          35|               85.71|
|2022-01-05|   anukriti|          35|               57.14|
+----------+-----------+------------+--------------------+

